In [ ]:
#@markdown # **1) Download required Libraries**
# # Clone Real-ESRGAN and enter the Real-ESRGAN
from google.colab import drive, files
import os, shutil, subprocess
drive_mounted = False
temp_folder = 'tmp_frames'
result_folder = 'results'
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr facexlib gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
%cd ..

In [ ]:
import os
from google.colab import drive
if os.path.exists('/content/drive') is not True:
  drive.mount('/content/drive')

drive_mounted = True
drive_path = 'test.mp4'
file_name = drive_path.split('/')[-1]
input_path = f'/content/drive/MyDrive/{drive_path}'
print("video path set:\n",input_path)

In [ ]:
#@title # 3) Extract Frames from Video

if os.path.isdir(temp_folder):
  shutil.rmtree(temp_folder)

os.mkdir(temp_folder)
print(f'Extracting Frames to: {temp_folder}')
cmd = [
       'ffmpeg',
       '-i',
       input_path,
       '-qscale:v',
       '1',
       '-qmin',
       '1',
       '-qmax',
       '1',
       '-vsync',
       '0',
       f'{temp_folder}/frame_%08d.png'
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    frame_count = len(os.listdir(f'/content/{temp_folder}'))
    print(f"Done, Extracted {frame_count} Frames")


In [ ]:
#@title # **4) Run ERSGAN on Extracted Frames**
%cd /content/Real-ESRGAN
frame_count = len(os.listdir(f'/content/{temp_folder}'))
print(f"Enhancing {frame_count} Frames with ESRGAN...")
cmd = [
       'python',
       'inference_realesrgan.py',
       '-n',
       'RealESRGAN_x4plus',
       '-i',
       f'../{temp_folder}',
       '--outscale',
       '4',
       '--face_enhance'
]
#process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# stdout, stderr = process.communicate()
# if process.returncode != 0:
#     print(stderr)
#     raise RuntimeError(stderr)
# else:
#     print("Done Extracting Frames")
!{' '.join(cmd)}
print("Done Enhancing Frames")
%cd ..

In [ ]:
#@title # **5) Recreate Video From Enhanced Frames**
import subprocess
frame_count = len(os.listdir(f'/content/{temp_folder}'))
if os.path.isdir(result_folder):
  shutil.rmtree(result_folder)
os.mkdir(result_folder)

fps = 15
print(f"Recompiling {frame_count} Frames into Video...")
cmd = [
       'ffmpeg',
       '-i',
       f'/content/Real-ESRGAN/results/frame_%08d_out.png',
       '-c:a',
       'copy',
       '-c:v',
       'libx264',
       '-r',
       str(fps),
       '-pix_fmt',
       'yuv420p',
       f'{result_folder}/enhanced_{file_name}'
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Done Recreating Video")

In [ ]:
#@title # **6) Download Super Enhanced Video**


from google.colab import files

filename= os.listdir(result_folder)
if drive_mounted:
  if os.path.exists('/content/drive/MyDrive/Hosts-AI-Edits/ESRGAN-4-Vid') is not True:
    os.makedirs('/content/drive/MyDrive/Hosts-AI-Edits/ESRGAN-4-Vid')
  shutil.move(f'{result_folder}/{filename[0]}',f'/content/drive/MyDrive/Hosts-AI-Edits/ESRGAN-4-Vid/{filename[0]}')
  print(f"moved {filename[0]} to the folder Hosts-AI-Edits/ESRGAN-4-Vid on your Google Drive")
else:
  files.download(f'{result_folder}/{filename[0]}')

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip


esrgan_video = VideoFileClip("/content/drive/MyDrive/Hosts-AI-Edits/ESRGAN-4-Vid/enhanced_test.mp4")
original_audio = AudioFileClip("/content/drive/MyDrive/test.mp4")
video_with_audio = esrgan_video.set_audio(original_audio)
video_with_audio.write_videofile("/content/drive/MyDrive/esrgan.mp4", codec='libx264')

In [ ]:
#@title # **7) Metrics**
import cv2
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity


original_video = cv2.VideoCapture("/content/drive/MyDrive/test.mp4")
esrgan_output_video = cv2.VideoCapture("/content/drive/MyDrive/esrgan.mp4")


fps = original_video.get(cv2.CAP_PROP_FPS)


frame_count = min(
    int(original_video.get(cv2.CAP_PROP_FRAME_COUNT)),
    int(esrgan_output_video.get(cv2.CAP_PROP_FRAME_COUNT))
)


psnr_values = []
ssim_values = []


for frame_index in range(frame_count):

    original_frame = original_video.read()[1]
    swinir_frame = esrgan_output_video.read()[1]


    if original_frame.shape != swinir_frame.shape:
        swinir_frame = cv2.resize(swinir_frame, (original_frame.shape[1], original_frame.shape[0]))


    psnr = peak_signal_noise_ratio(original_frame, swinir_frame)
    ssim = structural_similarity(original_frame, swinir_frame, multichannel=True)


    psnr_values.append(psnr)
    ssim_values.append(ssim)


average_psnr = sum(psnr_values) / len(psnr_values)
average_ssim = sum(ssim_values) / len(ssim_values)

print(f"Average PSNR: {average_psnr:.2f}")
print(f"Average SSIM: {average_ssim:.2f}")

In [ ]:
#@markdown # **Optional**: Unmount Google Drive
#@markdown might be useful if you decide you want to upload on here instead
drive_mounted = False
drive.flush_and_unmount()